In [1]:
# test

# Intro

team name

report location/background/info: Prepare a report containing your results from the analysis. It should contain the following: Intro, data cleaning/pre-processing, visualizations (at least 3), analysis/results, conclusion/future improvements.

In [41]:
# Imports
import pandas as pd
import numpy as np
import plotly
import plotly.express as px

In [42]:
# Remove automatic formatting with symbol usage (Ex: $ sign -> MathJax)
pd.options.display.html.use_mathjax=False

In [43]:
# Read training data
train_data = pd.read_csv('../data/advanced_trainset.csv')
train_data.head()

,Sentence,Sentiment
0,According to the Finnish-Russian Chamber of Co...,neutral
1,The Swedish buyout firm has sold its remaining...,neutral
2,$SPY wouldn't be surprised to see a green close,positive
3,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
4,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative


In [44]:
# Read testing data
test_data = pd.read_csv('../data/advanced_testset.csv')
test_data.head()

,Sentence
0,Earnings per share ( EPS ) dropped to EUR 0.21...
1,$SONC Amazing run since middle of March - obvi...
2,"Ruukki Romania , the local arm of Finnish meta..."
3,Self-service and automation are in a bigger ro...
4,Alma Media 's operating profit amounted to EUR...


In [48]:
# Read supplementary stock ticker data
stocks = pd.read_csv('../data/stock_tickers.csv')
stocks.head()

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,A,Agilent Technologies Inc. Common Stock,$134.87,-1.0600,-0.78%,4.047629e+10,United States,1999.0,2070939,Capital Goods,Electrical Products
1,AA,Alcoa Corporation Common Stock,$84.15,-1.9500,-2.265%,1.551901e+10,NaN,2016.0,4585478,Basic Industries,Metal Fabrications
2,AAC,Ares Acquisition Corporation Class A Ordinary ...,$9.83,0.0200,0.204%,1.228750e+09,NaN,2021.0,186747,Finance,Business Services
3,AACG,ATA Creativity Global American Depositary Shares,$1.21,-0.0600,-4.724%,3.796607e+07,China,NaN,7154,Miscellaneous,Service to the Health Industry
4,AACI,Armada Acquisition Corp. I Common Stock,$9.9781,0.1181,1.198%,2.066415e+08,United States,2021.0,174251,Consumer Durables,Consumer Electronics/Appliances


# EDA

Look through the dataset for things that catch your eye. What proportion of responses are negative, positive, and neutral? Do you see any imbalances in the data? What else do you find? Please provide charts and visualizations to support your claim.

In [25]:
sentiment_counts = train_data['Sentiment'].value_counts().to_frame().reset_index()
sentiment_counts

,index,Sentiment
0,neutral,2363
1,positive,1383
2,negative,636


In [26]:
fig = px.bar(sentiment_counts, x='index', y='Sentiment', \
             title="Sentiment Counts in Training Data", labels={'index':'Sentiment', 'Sentiment': 'Count'})
fig.show()

As we see from this bar chart, there is a significant imbalance in the number of observations we have for neutral, positive, and negative sentences. This will mean... TODO: HERE

## Negative Sentences

In [40]:
train_data[train_data['Sentiment'] == 'negative']['Sentence'].iloc[9]

"Finnish GeoSentric 's net sales decreased to EUR 939,000 in January-March 2009 ."

# Feature Engineering

Do you need to make any changes to “Sentence” to make it more digestible for your model? Will you make any restrictions to your sample? Even if you don’t choose to make any changes to the data, please describe your reasoning.

# Model Building

Create a NLP model that uses the “Sentence” as an input, using “Sentiment”  as labels. Ideally, you will compare the results of several different models to find the optimal choice. What led you to choose your final model? Did you run into any roadblocks? Please describe your process in depth. Make sure to train your model on the training set only.

# Model Testing

Please report the performance of your model on the training set. How does your model perform? Please report your accuracy and F1 score. Also, using the test set, please provide a CSV of your predicted values for “Sentiment” with your submission.